# Books to Scrape

### Importing the needed libraries 

In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from time import sleep
import re
import itertools

We create a list with the 5 categories that we want to study

In [46]:
list_categories = ['travel_2', 'religion_12', 'sports-and-games_17', 'business_35', 'politics_48']
list_categories_best = ['Travel', 'Religion', 'Sports and Games', 'Business', 'Politics']

### Creation of the function 

In [66]:
import pandas as pd

url_list = []
title = []
price = []
availability = []
description = []
category = []
star_rating = []
count_categ = []

for i in list_categories:
    url_list_count = [] # on créé une liste qui pour chaque catégorie va nous servir à compter le nb de livres associé
    url = "https://books.toscrape.com/catalogue/category/books/" + i + "/index.html"
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser') 
    ID_list = soup.find_all('a', href=True)
    for j  in ID_list : 
        if "../../../" in j.get("href"):
            url_list.append('https://books.toscrape.com/catalogue' + j.get('href').replace('../../..', ''))
            url_list_count.append('https://books.toscrape.com/catalogue' + j.get('href').replace('../../..', ''))
    url_list_count = list(dict.fromkeys(url_list_count))
    count_categ.append(len(url_list_count) - 1) # on ajoute le nb de livre pour chaque catégorie (-1 car à chaque fois il y a un titre en trop au début)
url_list = list(dict.fromkeys(url_list))  
del url_list[0] # on enlève l'url du début qui n'est pas un livre

for j in range(len(url_list)): # on se place à chaque fois dans un URL de la liste
    page = requests.get(url_list[j])
    soup = BeautifulSoup(page.content, 'html.parser') 
    
    # titre
    search_list = soup.find_all('h1', class_ = None)
    for i in search_list : 
        title.append(i.text)
        
    # prix
    search_list = soup.find_all('p', class_ = "price_color")
    search = search_list[0]
    search_sort = search.text.replace('£',' ')
    search_sort_float = float(search_sort)
    price.append(search_sort_float)
    
    # disponibilité
    search_list = soup.find_all('p', class_ = "instock availability")
    search = search_list[0]
    if 'In stock' in search.text:
        availability.append(1)
    else:
        availability.append(0)
        
    # description
    search_list = soup.find_all('p', class_ = None)
    for i in search_list : 
        description.append(i.text)
        
    # notation
    star = soup.find_all('div', attrs={'class': 'col-sm-6 product_main'})
    str_star=str(star)
    if 'star-rating One' in str_star:
        star_rating.append(1)
    elif 'star-rating Two' in str_star:
        star_rating.append(2)
    elif 'star-rating Three' in str_star:
        star_rating.append(3)
    elif 'star-rating Four' in str_star:
        star_rating.append(4)
    elif 'star-rating Five' in str_star:
        star_rating.append(5)

# création de la liste des catégories
for i in range(len(count_categ)):
    category.append([list_categories_best[i]] * count_categ[i]) # nom de la catégorie * nb de fois où elle apparait
category = list(itertools.chain(*category))       

df = pd.DataFrame(list(zip(title, category, price, availability, description, star_rating)), columns = ['Title', 'Category', 'Price', 'Availability', 'Description', 'Star Rating'])

### Display of the obtained dataset 

In [67]:
df

,Title,Category,Price,Availability,Description,Star Rating
0,It's Only the Himalayas,Travel,45.17,1,"“Wherever you go, whatever you do, just . . . ...",2
1,Full Moon over Noah’s Ark: An Odyssey to Mount...,Travel,49.43,1,Acclaimed travel writer Rick Antonson sets his...,4
2,See America: A Celebration of Our National Par...,Travel,48.87,1,To coincide with the 2016 centennial anniversa...,3
3,Vagabonding: An Uncommon Guide to the Art of L...,Travel,36.94,1,With a new foreword by Tim Ferriss •There’s no...,2
4,Under the Tuscan Sun,Travel,37.33,1,A CLASSIC FROM THE BESTSELLING AUTHOR OF UNDER...,3
5,A Summer In Europe,Travel,44.34,1,"On her thirtieth birthday, Gwendolyn Reese rec...",2
6,The Great Railway Bazaar,Travel,30.54,1,"First published more than thirty years ago, Pa...",1
7,A Year in Provence (Provence #1),Travel,56.88,1,National BestsellerIn this witty and warm-hear...,4
8,The Road to Little Dribbling: Adventures of an...,Travel,23.21,1,The hilarious and loving sequel to a hilarious...,1
9,Neither Here nor There: Travels in Europe,Travel,38.95,1,"Bill Bryson's first travel book, The Lost Cont...",3
